In [ ]:
import ast
import inspect
import math
import os
import sys
from io import StringIO

import matplotlib.pyplot as plt
import pandas as pd
import torch
from IPython.display import clear_output

from epsclassifiers.cr_chest_classifier import CrChestClassifier
from epsclassifiers.cr_projection_classifier import CrProjectionClassifier
from epsutils.dicom import dicom_utils
from epsutils.gcs import gcs_utils
from epsutils.image import image_utils

INPUT_FILE_NAME = "/mnt/training/splits/gradient_batches_1-5_segmed_batches_1-4_simonmed_batches_1-10_reports_with_labels_with_arm_segment_test.csv"
BASE_IMAGES_PATH = None
IMAGE_PATH_COLUMN_NAME = "image_paths"
COLUMN_NAME_FOR_NON_EMPTY_FILTERING = "arm_segment"
CSV_SEPARATOR = ","
CSV_HEADER = True
RUN_PROJECTION_CLASSIFICATION = False
RUN_CHEST_NON_CHEST_CLASSIFICATION = False
FIGURE_SIZE = 15
BASE_PATH_SUBSTITUTIONS = {
    "gradient/22JUL2024": None,
    "gradient/20DEC2024": None,
    "gradient/09JAN2025": None,
    "gradient/16AUG2024": "/mnt/sfs-gradient-nochest/16AUG2024",
    "gradient/13JAN2025": "/mnt/sfs-gradient-nochest/13JAN2025/deid",
    "segmed/batch1": "/mnt/sfs-segmed-1",
    "segmed/batch2": "/mnt/sfs-segmed-2",
    "segmed/batch3": "/mnt/sfs-segmed-34/segmed_3",
    "segmed/batch4": "/mnt/sfs-segmed-34/segmed_4",
    "simonmed": "/mnt/sfs-simonmed"
}

# Download or load input file.
if gcs_utils.is_gcs_uri(INPUT_FILE_NAME):
    print(f"Downloading input file {INPUT_FILE_NAME}")
    gcs_data = gcs_utils.split_gcs_uri(INPUT_FILE_NAME)
    content = gcs_utils.download_file_as_string(gcs_bucket_name=gcs_data["gcs_bucket_name"], gcs_file_name=gcs_data["gcs_path"])
else:
    print(f"Loading input file {INPUT_FILE_NAME}")
    with open(INPUT_FILE_NAME, "r") as file:
        content = file.read()

# Convert to Pandas dataset.
print("Converting input file to Pandas dataset")
if INPUT_FILE_NAME.endswith(".csv"):
    df = pd.read_csv(StringIO(content), low_memory=False, sep=CSV_SEPARATOR, header=0 if CSV_HEADER else None)
elif INPUT_FILE_NAME.endswith(".jsonl"):
    df = pd.read_json(StringIO(content), lines=True)
else:
    raise ValueError("Input file type not supported")

# Perform filtering.
if COLUMN_NAME_FOR_NON_EMPTY_FILTERING is not None:
    df = df[
        df[COLUMN_NAME_FOR_NON_EMPTY_FILTERING].notna() &
        df[COLUMN_NAME_FOR_NON_EMPTY_FILTERING].astype(bool)
    ]
    print(f"Number of rows after filtering: {len(df)}")

# Instantiate projection classifier.
if RUN_PROJECTION_CLASSIFICATION:
    print("Loading the projection classification model")
    MODEL_PATH = os.path.join(os.path.dirname(inspect.getmodule(CrProjectionClassifier).__file__), "models/cr_projection_classifier_trained_on_500k_gradient_samples.pt")
    projection_classifier = CrProjectionClassifier()
    projection_classifier.load_state_dict(torch.load(MODEL_PATH))

# Instantiate chest/non-chest classifier.
if RUN_CHEST_NON_CHEST_CLASSIFICATION:
    print("Loading the chest/non-chest classification model")
    MODEL_PATH = os.path.join(os.path.dirname(inspect.getmodule(CrChestClassifier).__file__), "models/cr_chest_classifier_trained_on_600k_gradient_samples.pt")
    chest_classifier = CrChestClassifier()
    chest_classifier.load_state_dict(torch.load(MODEL_PATH))

# Print command.
print("Commands:")
print("- row number (zero-based indexing) - go to the row number")
print("- '+' - go to the next row")
print("- '-' - go to the previous row")
print("- 'q' - quit")

row_number = -1

while True:
    selection = input("Enter command: ")
    clear_output(wait=True)

    try:
        selected_row_number = int(selection)
    except:
        selected_row_number = None

    if selected_row_number is not None:
        row_number = selected_row_number
    elif selection == "+":
        row_number += 1
    elif selection == "-":
        row_number -= 1
    elif selection == "q":
        break
    else:
        print("Invalid command")
        continue

    # Get row.
    row_number = max(0, min(row_number, len(df) - 1))
    row = df.iloc[row_number]

    # Load image data.
    print("Loading image data...")
    sys.stdout.flush()

    try:
        image_paths = ast.literal_eval(row[IMAGE_PATH_COLUMN_NAME])
    except:
        image_paths = row[IMAGE_PATH_COLUMN_NAME]

    image_paths = image_paths if isinstance(image_paths, list) else [image_paths]

    if BASE_IMAGES_PATH is not None:
        image_paths = [os.path.join(BASE_IMAGES_PATH, image_path) for image_path in image_paths]

    if "base_path" in df.columns and BASE_PATH_SUBSTITUTIONS is not None:
        base_path = row["base_path"]
        if base_path not in BASE_PATH_SUBSTITUTIONS:
            print("Base path not in base path substitutions")
            continue
        elif BASE_PATH_SUBSTITUTIONS[base_path] is None:
            print("Substitution for the given base path is None")
            continue
        subst = BASE_PATH_SUBSTITUTIONS[base_path]
        image_paths = [os.path.join(subst, image_path) for image_path in image_paths]

    numpy_images = [dicom_utils.get_dicom_image_fail_safe(image_path, custom_windowing_parameters={"window_center": 0, "window_width": 0}) for image_path in image_paths]

    if RUN_PROJECTION_CLASSIFICATION:
        projections = projection_classifier.predict(images=numpy_images, device="cuda")
    else:
        projections = None

    if RUN_CHEST_NON_CHEST_CLASSIFICATION:
        chest_classification_results = chest_classifier.predict(images=numpy_images, device="cuda")
    else:
        chest_classification_results = None

    images = [image_utils.numpy_array_to_pil_image(numpy_image) for numpy_image in numpy_images]

    # Show info.
    print(f"Row number: {row_number}")

    for column, value in row.items():
        if isinstance(value, str):
            value = value.replace("\n", " ")
        print(f"{column}: {value}")

    if projections:
        print(f"Projections: {projections}")

    if chest_classification_results:
        print(f"Chest/non-chest classification: {chest_classification_results}")

    # Show image data.
    if len(images) == 1:
        plt.imshow(images[0])
    else:
        num_images = len(images)
        cols = 2
        rows = math.ceil(num_images / cols)
        fig, axs = plt.subplots(rows, cols, figsize=(FIGURE_SIZE, FIGURE_SIZE))
        for i in range(num_images):
            plt.subplot(rows, cols, i + 1)
            plt.imshow(images[i])
            plt.axis("off")
    plt.tight_layout()
    plt.show()

print("Finished")